In [14]:
import json
import pandas as pd
import pymongo

In [6]:
# Establish a connection to the MongoDB server
client = pymongo.MongoClient("mongodb+srv://@airbnb.vsxcnz2.mongodb.net/")

# Access the 'Airbnb' database
db = client["airbnb"]

# Access the 'airbnb' collection
col = db["airbnb_analysis"]

In [7]:
# Extract Rooms info and transformed into dataframe

def Room_info():
    
    room_data=[]

    for i in col.find():
        data=dict(
            id=i['_id'],
            images=i['images']['picture_url'],
            property_type=i['property_type'],
            room_type=i['room_type'],
            bed_type=i['bed_type'],
            minimum_nights=i['minimum_nights'],
            maximum_nights=i['maximum_nights'],
            accommodates=i['accommodates'],
            bedrooms=i.get('bedrooms'),
            beds=i.get('beds'),
            bathrooms=i.get('bathrooms'),
            amenities=i['amenities'],
            guests_included=i['guests_included'],
            availability_30=i['availability']['availability_30'],
            availability_60=i['availability']['availability_60'],
            availability_90=i['availability']['availability_90'],
            availability_365=i['availability']['availability_365']
            )
        room_data.append(data)

    Room_info=pd.DataFrame(room_data)
    Room_info[['bedrooms','beds','bathrooms']]=Room_info[['bedrooms','beds','bathrooms']].fillna(0)
    Room_info[['bedrooms','beds','bathrooms','guests_included']]=Room_info[['bedrooms','beds','bathrooms','guests_included']].astype(dtype='int64')
    Room_info['amenities'] = [', '.join(map(str, x)) if isinstance(x, list) else '' for x in Room_info['amenities']]
    
    return Room_info


In [8]:
# Extract Hotels info and transformed into dataframe

def Hotels_info():
        
        hotel_data=[]

        for i in col.find():
                data=dict(
                        id=i['_id'],
                        listing_url=i['listing_url'],
                        name=i['name'],
                        description=i['description'],
                        neighborhood_overview=i['neighborhood_overview'],
                        transit=i['transit'],
                        street=i['address']['street'],
                        suburb=i['address']['suburb'],
                        government_area=i['address']['government_area'],
                        market=i['address']['market'],
                        country=i['address']['country'],
                        country_code=i['address']['country_code'],
                        coordinates=i['address']['location'].get('coordinates'),
                        price=i['price'],
                        weekly_price=i.get('weekly_price'),
                        monthly_price=i.get('monthly_price'),
                        security_deposit=i.get('security_deposit'),
                        cleaning_fee=i.get('cleaning_fee'),
                        cancellation_policy=i['cancellation_policy'],
                        )
                hotel_data.append(data)

        Hotel_info=pd.DataFrame(hotel_data)
        Hotel_info[['security_deposit','cleaning_fee','weekly_price','monthly_price']]=Hotel_info[['security_deposit','cleaning_fee','weekly_price','monthly_price']].fillna(0)
        Hotel_info[['price','security_deposit','cleaning_fee','weekly_price','monthly_price']]=Hotel_info[['price','security_deposit','cleaning_fee','weekly_price','monthly_price']].astype(dtype='int64')
        Hotel_info['coordinates'] = [', '.join(map(str, x)) if isinstance(x, list) else '' for x in Hotel_info['coordinates']]

        
        return Hotel_info

In [9]:
# Extract Hosts info and transformed into dataframe

def Host_info():

    host_data=[]

    for i in col.find():
        data=dict(
            id=i['_id'],
            host_id=i['host']['host_id'],
            host_url=i['host']['host_url'],
            host_name=i['host']['host_name'],
            host_location=i['host']['host_location'],
            host_about=i['host']['host_about'],
            host_picture_url=i['host']['host_picture_url'],
            host_neighbourhood=i['host']['host_neighbourhood'],
            host_response_rate=i['host'].get('host_response_rate'),
            host_total_listings=i['host']['host_total_listings_count'],
            host_verifications=i['host']['host_verifications']
            )
        host_data.append(data)

    Host_info=pd.DataFrame(host_data)
    Host_info['host_response_rate']=Host_info['host_response_rate'].fillna(0).astype(dtype='int64')
    Host_info['host_verifications'] = [', '.join(map(str, x)) if isinstance(x, list) else '' for x in Host_info['host_verifications']]
    

    return Host_info

In [10]:
# Extract Reviews info and transformed into dataframe

def Review_info():

    review_data=[]

    for i in col.find():
        data=dict(
            id=i['_id'],
            first_review=i.get('first_review'),
            last_review=i.get('last_review'),
            number_of_reviews=i['number_of_reviews'],
            overall_score=i['review_scores'].get('review_scores_accuracy'),
            cleanliness_score=i['review_scores'].get('review_scores_cleanliness'),
            checkin_score=i['review_scores'].get('review_scores_checkin'),
            communication_score=i['review_scores'].get('review_scores_communication'),
            location_score=i['review_scores'].get('review_scores_communication'),
            value_score=i['review_scores'].get('review_scores_value'),
            rating=i['review_scores'].get('review_scores_rating')
        )
        review_data.append(data)

    Review_info=pd.DataFrame(review_data)
    Review_info[['first_review','last_review']]=Review_info[['first_review','last_review']].fillna('Not Available')
    Review_info[['overall_score','cleanliness_score','checkin_score','communication_score','location_score','value_score','rating']]=Review_info[['overall_score','cleanliness_score','checkin_score','communication_score','location_score','value_score','rating']].fillna(0).astype(dtype='int64')

    return Review_info

In [16]:
# Extract Review commands info and transformed into dataframe

def Review_comments_info():

    comments_data=[]

    for j in col.find():
        r_list=j.get('reviews',[])
        for i in r_list:
            data=dict(
                id=i.get('listing_id'),
                comment_id=i.get('_id'),
                date=i.get('date'),
                reviewer_id=i.get('reviewer_id'),
                reviewer_name=i.get('reviewer_name'),
                comments=i.get('comments')
                )
            comments_data.append(data)

    Review_comments_info=pd.DataFrame(comments_data)
    Review_comments_info[['reviewer_name','comments']]=Review_comments_info[['reviewer_name','comments']].fillna('Not Available')

    return Review_comments_info

In [17]:
# Merge the DataFrames on the common 'id' column
hotels_df = Hotels_info()
rooms_df = Room_info()
hosts_df = Host_info()
reviews_df = Review_info()
comments_df = Review_comments_info()

# Merging DataFrames
combined_df = hotels_df.merge(rooms_df, on='id').merge(hosts_df, on='id').merge(reviews_df, on='id').merge(comments_df, on='id', how='left')

# Save the merged DataFrame to a CSV file
combined_df.to_csv('airbnb_combine_data3.csv', index=False)